In [2]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')

pd.set_option('display.max_colwidth', None)

In [3]:
import sys
import os
sys.path.append('../src-py/')
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from ca_utils import *

2022-08-12 10:03:56,075 loading file ../../data-ceph/arguana/arg-generation/claim-target-tagger/model/final-model.pt
2022-08-12 10:04:25,321 SequenceTagger predicts: Dictionary with 4 tags: <unk>, B-CT, I-CT, O


In [4]:
torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device(type='cuda')

In [15]:
path_to_arg_quality_model='../sample-data/argument-quality-model'
data_path = '../sample-data/'

In [9]:
from transformers import BertForSequenceClassification, AutoTokenizer, TextClassificationPipeline

#take top quality counter from the dataset!
gretz_model = BertForSequenceClassification.from_pretrained(path_to_arg_quality_model, local_files_only=True, cache_dir='cache')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir='cache')

arg_quality_pipeline = TextClassificationPipeline(model=gretz_model, tokenizer=tokenizer, framework='pt', task='ArgQ', device=0)

def get_best_argumentative_comment(comments):    
    comments = [' '.join(x) for x in comments]
    # set the pipeline
    results = arg_quality_pipeline(comments, truncation=True)
    comments_scores = []
    for result_dict in results:
        comments_scores.append(result_dict['score'])
    
    comments = sorted(list(zip(comments, comments_scores)), key= lambda x: -x[1])
    return comments[0][0]

### Preparing the data:
- Starting from jo_data, remove all cases where the comment qoutes the post --> considered undermining
- Second remove bot comments
- Third, split into training, testing, and validation
- For the training process further to only select the comments with good argumentative quality
- ..

In [11]:
rebuttal_df = pd.read_pickle("../sample-data/sample_argument_rebuttals.pkl")

In [12]:
#First save all the data without masking..
train_df = rebuttal_df[rebuttal_df['split'] == 'train']
dev_df   = rebuttal_df[rebuttal_df['split'] == 'val']
test_df  = rebuttal_df[rebuttal_df['split'] == 'test']

In [13]:
train_df.post_id.nunique()

1094

In [14]:
dev_df.post_id.nunique()

413

#### Removing the undermining-cases:

The code for this in the arg-undermining project

#### Removing bot comments:

In [ ]:
#1.Remove all cases where there is a bot comment
df['bot_comment'] = df.comment.apply(lambda x:  'i am a bot, and this action was performed automatically.' in ' '.join(x))
df = df.drop(df[df.bot_comment].index)

#### Split the corpus:

In [16]:
#First save all the data without masking..
train_df = rebuttal_df[rebuttal_df['split'] == 'train']
dev_df   = rebuttal_df[rebuttal_df['split'] == 'val']
test_df  = rebuttal_df[rebuttal_df['split'] == 'test']

In [17]:
dev_df.to_pickle(data_path + '/valid_conclusion_all.pkl')
test_df.to_pickle(data_path + '/test_conclusion_all.pkl')

#Also save a sample for rapid test
dev_df.drop_duplicates('post_id').sample(200).to_pickle(data_path  + '/sample_valid_conclusion_all.pkl')
test_df.drop_duplicates('post_id').sample(200).to_pickle(data_path + '/sample_test_conclusion_all.pkl')

#### Prepare the training data:

##### Select high quality comments to be the counters

In [18]:
post_to_comments_df = train_df.groupby('post_id').agg({'counter': lambda x: list(x)}).reset_index()
post_to_comments_df['best_comment'] = post_to_comments_df.counter.apply(lambda x: get_best_argumentative_comment(x))
post_to_best_comment = pd.Series(post_to_comments_df.best_comment.values,index=post_to_comments_df.post_id).to_dict()

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [19]:
train_df['counter'] = train_df.post_id.apply(lambda x: post_to_best_comment[x]) #override the comments with the best one...

<ipython-input-19-631f10eb3b57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['counter'] = train_df.post_id.apply(lambda x: post_to_best_comment[x]) #override the comments with the best one...


In [21]:
train_df.drop_duplicates('post_id').to_pickle(data_path + '/preprocessed_train_conclusion_all.pkl')